# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment, Dataset, Model
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
import numpy as np
import joblib, pickle

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [4]:
subscription_id = '8a56ebc5-caa7-4a43-b8ed-1c1895e46051'
resource_group = 'aml-quickstarts-128350'
workspace_name = 'quick-starts-ws-128350'

ws = Workspace.get(name="quick-starts-ws-128350")
exp = Experiment(workspace=ws, name="Ranga")

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='mpg')
ds = dataset.to_pandas_dataframe()

ds = ds.replace('?',0)
ds.dropna(inplace=True)

ds.drop(columns=['car name'],inplace=True)

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
# TODO: Put your automl config here
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='regression',
    primary_metric='r2_score',
    training_data=ds,
    label_column_name='mpg',
    n_cross_validations=3)

In [7]:
# TODO: Submit your experiment
autoexp = Experiment(workspace=ws,name='AutoRanga')
remote_run = autoexp.submit(automl_config)
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       DONE
DESCRIPTION:  High cardinality features were detected in your inputs and handled.
              Learn more about high cardinality feature handling: https://aka.ms/AutomatedMLFeaturization
DETAILS:      High cardinality features refer to columns that contain a large percentage of unique values.
+---------------------------------+---------------------------------+
|Column name                      |Column Content Type              |
+=============================

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…


****************************************************************************************************

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:00:28       0.7822    0.7822
         1   MaxAbsScaler XGBoostRegressor                  0:00:25       0.8192    0.8192
         2   StandardScalerWrapper XGBoostRegressor         0:00:31       0.8308    0.8308
         3   StandardScalerWrapper RandomForest             0:00:30      

{'runId': 'AutoML_e5d14277-9d0c-49e7-bfd1-210e804b796c',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2020-11-29T14:12:09.831367Z',
 'endTimeUtc': '2020-11-29T14:33:20.451133Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'r2_score',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"AutoRanga","subscription_id":"8a56ebc5-caa7-4a43-b8ed-1c1895e46051","resource_group":"aml-quickstarts-128350","workspace_name":"quick-starts-ws-128350","region":"southcentralus","compute_target":"local","spark_service":null,"azure_service":"Microsoft.AzureNotebookVM","_local_managed_run_id":null,"many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"r2_score","task_type":"regression","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [8]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [9]:
best,fitted = remote_run.get_output()
print(best)

Run(Experiment: AutoRanga,
Id: AutoML_e5d14277-9d0c-49e7-bfd1-210e804b796c_44,
Type: None,
Status: Completed)


In [10]:
print(fitted)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [11]:
#TODO: Save the best model
filename = 'best_automl_model.sav'
# pickle.dump(fitted, open(filename, 'wb'))

joblib.dump(fitted,filename)

['best_automl_model.sav']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [12]:
registeredfinal = Model.register(workspace=ws,model_path='./best_automl_model.sav',model_name='registered.sav')

Registering model registered.sav


In [13]:
path = Model.get_model_path('registered.sav')
joblib.load(path)

RegressionPipeline(pipeline=Pipeline(memory=None,
                                     steps=[('datatransformer',
                                             DataTransformer(enable_dnn=None,
                                                             enable_feature_sweeping=None,
                                                             feature_sweeping_config=None,
                                                             feature_sweeping_timeout=None,
                                                             featurization_config=None,
                                                             force_text_dnn=None,
                                                             is_cross_validation=None,
                                                             is_onnx_compatible=None,
                                                             logger=None,
                                                             observer=None,
                                         

In [20]:
#Create inference config
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

env = Environment.get(workspace=ws,name='AzureML-AutoML')

inferencefinal = InferenceConfig(entry_script='score_11_28.py',environment=env) 

#Deploy model and check service state
from azureml.core.webservice import LocalWebservice, AciWebservice


deployfinal = AciWebservice.deploy_configuration()
svcs = Model.deploy(workspace=ws,name='deploy',models=[registeredfinal],inference_config=inferencefinal,deployment_config=deployfinal)
svcs.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running.......................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [21]:
print(svcs.state)

Healthy


In [22]:
print(svcs.scoring_uri)

http://b5b4e7e1-eac3-4d99-afbd-85bb9cd2a99f.southcentralus.azurecontainer.io/score


TODO: In the cell below, send a request to the web service you deployed to test it.

In [24]:
import requests
import json

# URL for the web service
scoring_uri = 'http://b5b4e7e1-eac3-4d99-afbd-85bb9cd2a99f.southcentralus.azurecontainer.io/score'


data = {'data':[{
    'cylinders':4,
    'displacement':84,
    'horsepower':65,
    'weight':2110,
    'acceleration':12,
    'model yr':80,
    'origin':3}
]
        }

input_data = json.dumps(data)

headers = {'Content-Type': 'application/json'}


# # Make the request and display the response
result = requests.post(scoring_uri,input_data,headers=headers)
print("prediction:",result.text)

prediction: [37.301090683637064]


TODO: In the cell below, print the logs of the web service and delete the service

In [25]:
print(svcs.get_logs())

2020-11-29T14:57:52.1043349Z stdout F 2020-11-29T14:57:52,102127200+00:00 - gunicorn/run 
2020-11-29T14:57:52.2234545Z stdout F 2020-11-29T14:57:52,222557700+00:00 - rsyslog/run 
2020-11-29T14:57:52.4139565Z stdout F 2020-11-29T14:57:52,412558400+00:00 - iot-server/run 
2020-11-29T14:57:52.8038284Z stdout F 2020-11-29T14:57:52,802301400+00:00 - nginx/run 
2020-11-29T14:57:53.0168163Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-29T14:57:53.0168163Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-29T14:57:53.0208329Z stderr F /usr/sbin/nginx: /azureml-envs/azureml_7cf2b5eebfc9a832d561ce064cfbc38b/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2020-11-29T14:57:53.0208329Z stderr F /usr/sbin/nginx:

In [ ]:
svcs.delete()